In [18]:
import numpy as np


def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))


def sigmoid_backward(A):
    return A * (1 - A)


def relu(Z):
    return np.maximum(0, Z)


def relu_backward(Z):
    return 
    

def cost(Yh, Y):
    """
    A -- numpy array of size (1, number of examples)
    Y -- label vector of size (1, number of examples)
    """
    return (np.dot(Y, np.log(Yh).T) + np.dot((1 - Y), np.log(1 - Yh).T))/-Y.shape[1]


def initialize_parameters(layer_dims):
    parameters = {}
    
    for l in range(1, len(layer_dims)):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters
        

def activatef(activation, direction='forward'):
    if activation == 'sigmoid':
        return sigmoid if direction == 'forward' else sigmoid_backward
    else if activation == 'relu':
        return relu if direction == 'forward' else relu_backward
    

def linear_forward_propagate(parameters, X, layer_dims, activation='relu'): 
    A = X
    m = X.shape[1]
    
    caches = []
    L = len(layer_dims)
    
    activate = activatef(activation)
    for l in range(1, L):
        Z = np.dot(parameters['W' + str(l)], A) + parameters['b' + str(l)]
        if (l != L - 1):
            A = activate(Z)
        else:
            A = sigmoid(Z)            
        caches.append((Z, A))
    
    return A, caches


def backward_propagate(cache, Ap, dA, activation='relu'):
    Z, A = cache
    dZ = dA * activate_backward(Z)
    dW = np.dot(dZ, Ap.T)/m
    db = np.sum(dZ, axis=1, keepdims=True)/m
    dAp = np.dot(W.T, dZ) 
    
    grad = {'dZ': dZ, 'dW': dW, 'db': db}
    return dAp, grad

    
def linear_backward_propagate(parameters, caches, X, Y, layer_dims, activation='relu'):    
    L = len(layer_dims)
    current_cache = caches[L-1]
    A = current_cache[1]
    dAL = -np.divide(Y, A) + np.divide(1 - Y, 1 - A)
    dZL = A - Y
    grads['dA' + str(L)] = dAL
    grads['dW' + str(L)] = np.dot(dZL, )
    grads['db' + str(L)]    
    
    activate = activatef(activation)
    for l in reversed(range(1, L-1)):
        current_cache = caches[l]
        grads['dA' + str(l)] = 
        grads['dW' + str(l)]
        grads['db' + str(l)]
        
    
    dZ = A - Y
    dW = np.dot(dZ, Ap.T)/m
    db = np.sum(dZ, axis=1, keepdims = True)/m
    
    grad = {'dZ': dZ, 'dW': dW, 'db': db}
    return grad
    

def backward_propagate(A, Ap, dA):     
    m = A.shape[1]
    
    dZ = dA * dActivate(A)
    dW = np.dot(dZ, Ap.T)/m
    db = np.sum(dZ, axis=1, keepdims = True)/m
    
    grad = {'dZ': dZ, 'dW': dW, 'db': db}
    return grad


def nn_model(X, Y, hidden_nodes=(3, 3), learning_rate=0.01, iter=10000, print_th_iter=100):
    """
    X -- data of size (n, number of examples)
    Y -- true "label" vector of size (1, number of examples)
    """
    for i in range(iter):        
        for nl in hidden_nodes:
            W = np.random.randn() * 0.01


def accuracy(W, b, X, Y):
    p = predict(W, b, X)
    return 100 - np.sum(np.abs(p - Y))/Y.shape[1] * 100


In [19]:
np.random.seed(1)
training_set_X = np.random.randn(10, 5)
training_set_Y = np.array([[1, 0, 0, 1, 1]])
test_set_X = np.random.randn(10, 5)
test_set_Y = np.array([[1, 0, 0, 1, 1]])

layer_dims = [10, 3, 4, 2]

parameters = initialize_parameters(layer_dims)
forward_propagate(parameters, training_set_X, layer_dims, 'relu')

(array([[ 0.62394827,  0.77776369,  0.78530605,  0.64968868,  0.64189717],
        [ 0.55535721,  0.73735732,  0.74434649,  0.62457079,  0.56371896]]),
 [(array([[-2.98803784,  1.21657403, -2.95782519, -2.51056797, -1.98359606],
          [-0.69015729,  1.26078139,  2.17984754,  1.03822216, -2.05231958],
          [ 1.19263144, -6.37364929, -2.69631444, -4.63246047,  1.37462599]]),
   array([[ 0.        ,  1.21657403,  0.        ,  0.        ,  0.        ],
          [ 0.        ,  1.26078139,  2.17984754,  1.03822216,  0.        ],
          [ 1.19263144,  0.        ,  0.        ,  0.        ,  1.37462599]])),
  (array([[-1.46700601, -0.98151483, -1.3164549 , -0.62700378, -1.69086988],
          [-0.74364235,  1.66932577,  1.7281981 ,  0.82310966, -0.85712155],
          [ 0.95632468, -0.80944468, -2.49448977, -1.18808059,  1.10225901],
          [-0.12134533, -0.17857113, -0.40669366, -0.19370087, -0.13986252]]),
   array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.     

In [3]:
import os 
import imageio
import matplotlib.pyplot as plt
import skimage.transform as imgxf

%matplotlib inline


def load_image(img_path, width, height):
    img = np.array(imageio.imread(img_path))
    return imgxf.resize(img, (width, height), mode='reflect').reshape((1, width*height*3)).T, img
    

def load_image_X(folder, width, height):
    fname = np.array([[f] for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]).T
    img_X = np.zeros((width*height*3, fname.shape[1]))
    it = np.nditer(fname, flags=['f_index'])
    while not it.finished:
        print("Loading : ", it[0])
        fpath = os.path.join(folder, str(it[0]))
        x, img = load_image(fpath, width, height)
        img_X[:, it.index] = x[:,0]
        it.iternext()
    
    return fname, img_X
    

num_px = 300
image_folder = "data/images/22-80"
image_path = image_folder + '/22-80-01.jpg'

files, image_X = load_image_X(image_folder, num_px, num_px)
print("Files: {}, image: {}".format(files.shape, image_X.shape))

FileNotFoundError: [Errno 2] No such file or directory: 'data/images/22-80'

In [4]:
import numpy as np

a = np.random.randn(1, 1)
b = np.random.randn(1, 3)
print(a)
print(b)
np.dot(a, b)

[[-1.73293708]]
[[ 0.40687633  0.68986766  1.471552  ]]


array([[-0.70509109, -1.19549725, -2.55010703]])

In [2]:
import numpy as np
np.relu(1)

AttributeError: module 'numpy' has no attribute 'relu'